In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from underwriting import inventory_freshness_analyzer as ifa
from underwriting import inventory_freshness_analysis_query as ifa_query
import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
today = date.today()rea

In [8]:
query_join = ifa_query.create_inventory_days_since_package_metric_query(
    ['EL', '99HT'], ['C9-0000370-LIC', 'C10-0000279-LIC'], True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_join, engine).head()
#print(query_join)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C9-0000370-LIC,0184087030,active,Consumer,2022-04-30 20:54:15+00:00,2022-04-01 00:00:00+00:00,2,90.0,active,24704623,1A4060300022B79000320198,"BBrand Strawberry 1g Vape, Hashtag Distributio...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704623,1A4060300022B79000320198,active,2022-04-28,2022-05-01 11:18:24+00:00,Product,"BBrand Strawberry 1g Vape, Hashtag Distributio...",Vape Cartridge (weight - each),23.0,Each,3389924,Concentrate,,21422BDTSTR,"""33"" 12-12-19, #19, #20, #21, #22, #24, #25, #...",False,False,False,None,None,2,3
1,C9-0000370-LIC,0184086989,active,Consumer,2022-04-30 16:29:57+00:00,2022-04-01 00:00:00+00:00,1,65.0,active,24704654,1A4060300022B79000319194,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),Each,1.0,65.0,65.0,24704654,1A4060300022B79000319194,active,2022-04-28,2022-05-01 11:18:22+00:00,Product,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),23.0,Each,3205886,Concentrate,,13122LDSCAP25DUO,"Amnesia FF, Harvest FF, PY-09.30.20-AG0671, PY...",False,False,False,None,None,2,3
2,C9-0000370-LIC,0184086820,active,Consumer,2022-04-30 12:01:17+00:00,2022-04-01 00:00:00+00:00,2,120.0,active,24704294,1A4060300022B79000319219,Blue Dream LR Liquid Diamonds 1g 510 Vape Cart...,Vape Cartridge (weight - each),Each,1.0,45.0,45.0,24704294,1A4060300022B79000319219,active,2022-04-28,2022-05-01 11:18:20+00:00,Product,Blue Dream LR Liquid Diamonds 1g 510 Vape Cart...,Vape Cartridge (weight - each),34.0,Each,4425861,Concentrate,,32422BLUDREDUO,Lifted Organics- Blue Dream- 10.7.21,False,False,False,None,None,2,3
3,C9-0000370-LIC,0184087087,active,Consumer,2022-04-30 21:45:25+00:00,2022-04-01 00:00:00+00:00,1,70.0,active,24704579,1A4060300022B79000320203,"BBrand Apricot 1g Vape, Hashtag Distribution (...",Vape Cartridge (weight - each),Each,2.0,70.0,35.0,24704579,1A4060300022B79000320203,active,2022-04-28,2022-05-01 11:18:25+00:00,Product,"BBrand Apricot 1g Vape, Hashtag Distribution (...",Vape Cartridge (weight - each),9.0,Each,3389926,Concentrate,,32122BDTAPR,"1-21, 10-15 peanut butter souffle, 10-6 cherry...",False,False,False,None,None,2,3
4,C9-0000370-LIC,0184087049,active,Consumer,2022-04-30 21:53:20+00:00,2022-04-01 00:00:00+00:00,2,55.0,active,24704579,1A4060300022B79000320203,"BBrand Apricot 1g Vape, Hashtag Distribution (...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704579,1A4060300022B79000320203,active,2022-04-28,2022-05-01 11:18:25+00:00,Product,"BBrand Apricot 1g Vape, Hashtag Distribution (...",Vape Cartridge (weight - each),9.0,Each,3389926,Concentrate,,32122BDTAPR,"1-21, 10-15 peanut butter souffle, 10-6 cherry...",False,False,False,None,None,2,3


In [9]:
query_metric = ifa_query.create_company_freshness_metric_query(['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE, 'tx_product_category_name')
print(query_metric)


        SELECT 
            tx_product_category_name,
            AVG(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS avg_days_since_sale,
            MAX(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS max_days_since_sale,
            MIN(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS min_days_since_sale,
            COUNT(tx_product_category_name) AS number_of_sales,
            AVG(price_per_unit) AS avg_sale_price
        FROM 
            (
        SELECT 
            sales.license_number,
            sales.receipt_number,
            sales.rt_type,
            sales.sales_customer_type,
            sales.sales_datetime,
            sales.sales_month,
            sales.total_packages,
            sales.rt_total_price,
            sales.tx_type,
            sales.tx_package_id,
            sales.tx_package_label,
            sales.tx_product_name,
            sales.tx_product_category_name,
            sales.tx_unit_of_measure

In [10]:
pd.read_sql_query(query_metric, engine)

,tx_product_category_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,Pre-Roll Flower,100.636704,264,5,267,35.625468
1,Vape Cartridge (volume - each),30.641892,323,2,148,40.026486
2,Flower (packaged quarter - each),60.661972,120,5,71,72.225352
3,Flower (packaged ounce - each),98.666667,137,33,3,226.666667
4,Capsule (weight - each),112.566667,281,2,30,5.480000
5,Extract (weight - each),105.798817,386,2,169,30.546036
6,Other Concentrate (weight - each),121.642857,434,4,42,34.670476
7,Tincture (volume - each),404.461538,512,76,13,46.006154
8,Flower (packaged gram - each),43.121212,86,8,33,1.000000
9,Edible (volume - each),3.000000,4,2,2,1.000000


In [11]:
query_metric = ifa_query.create_company_freshness_metric_query(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_category_name')
pd.read_sql_query(query_metric, engine)

,tx_product_category_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,Vape Cartridge (volume - each),34.536232,323,1,897,51.063735
1,Other Concentrate (volume - each),33.793103,55,4,29,29.896552
2,Pre-Roll Infused,45.546490,224,0,2108,28.218082
3,Flower (packaged - each),40.407484,306,1,481,54.074324
4,Tincture (weight - each),11.000000,13,7,8,75.000000
5,Topical (volume - each),96.375000,432,4,176,60.728750
6,Capsule (weight - each),93.242857,281,0,70,44.562857
7,Pre-Roll Flower,51.160860,415,0,3071,19.987458
8,Shake (Packaged Gram - each),14.762712,44,2,59,46.610169
9,Flower (packaged quarter - each),38.316940,120,4,183,84.193989


In [12]:
query_metric = ifa_query.create_company_freshness_metric_query(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_name')
pd.read_sql_query(query_metric, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,SF-GDP-I-F-14.0,8.000000,15,3,11,175.0000
1,Defonce Dark Chocolate Bar 100MGTHC,97.938889,150,40,180,22.6300
2,ABX LIVE Dart Cartridge Grape Punch (.5g),41.583333,69,8,24,44.8125
3,14296 Kikoko HoneyShot Single Snooze 1:1 CBN 2...,107.750000,210,46,12,10.0000
4,"Kaneh Co SOLOS Salted Toffee Blondies 10mg, 1pc",41.000000,118,6,25,4.8400
...,...,...,...,...,...,...
690,Kurvana Originals Vape Cartridge 0.5ml Pineapp...,10.000000,15,7,3,40.0000
691,BeMore Blunt Gelato 41 x Watermelon Zkittles (...,123.000000,123,123,1,30.0000
692,YULA - Kief - Mango Sherbet,8.500000,11,6,2,25.0000
693,STIIIZY - Biscotti Pod - 1g,11.000000,11,11,1,55.0000


In [13]:
query_valuation = ifa_query.create_company_inventory_valudation_by_last_sale_price(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_valuation, engine)

,total_inventory_valuation
0,116445.76


In [14]:
query_valuation = ifa_query.create_company_inventory_valudation_by_last_sale_price(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, False)
pd.read_sql_query(query_valuation, engine)

,total_inventory_valuation
0,65669.76


In [15]:
query_top_product_by_speed = ifa_query.best_selling_products_by_liquidity(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_name',
    'avg_days_since_sale', 'ASC', '10')
pd.read_sql_query(query_top_product_by_speed, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,State Flower - Jarred Flower - 3.5 gram - Gela...,4.964286,9,1,28,70.000000
1,19023 OASIS Chocolate Peppermint Patties 100mg...,5.000000,8,1,19,24.802632
2,MVN - Blueberry Skittles - Eighth -PP,5.000000,9,1,24,69.416667
3,Source Flower Wookies Quarter 7g,6.000000,11,4,12,94.208333
4,"Eureka Kira Kira Diamond Infused Pre-Roll, Sun...",6.608696,10,3,23,55.000000
5,SF-Lambsbread-S-F-14.0,6.909091,14,1,11,173.181818
6,"Nightcap Kira Kira Diamond Infused Pre-Roll, S...",6.937500,10,3,16,55.000000
7,SF-GDP-I-F-14.0,8.000000,15,3,11,175.000000
8,Indoor - Connected - Eighth - Pantera Limone,8.538462,17,3,13,74.423077
9,Indoor - Connected - Pre-Roll Single - Pantera...,9.125000,18,3,16,18.000000


In [16]:
query_top_product_by_price = ifa_query.best_selling_products_by_liquidity(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, 'tx_product_name',
    'avg_sale_price', 'DESC', '10')
pd.read_sql_query(query_top_product_by_price, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,budbud Cosmic Glue 14g,37.595238,64,7,42,193.607143
1,Claybourne Co. Premium Small Bud 14g - (Case o...,45.472222,70,12,36,184.763889
2,Day Dreams Pouch (Sativa) Crescendo 14g,30.522727,66,6,44,183.738636
3,SF-GDP-I-F-14.0,8.000000,15,3,11,175.000000
4,SF-Lambsbread-S-F-14.0,6.909091,14,1,11,173.181818
5,Claybourne Co. Premium Small Bud 14g - (Case o...,19.555556,36,2,27,164.814815
6,budbud Dragonfruit 14g,62.109091,125,2,55,162.754545
7,"MR. ZIPS | Sunshine Punch | 14g Flower, Fort P...",20.291667,40,5,24,155.000000
8,MM-FloWhite-F-100T-14.18,74.466667,133,3,30,147.483333
9,Orange Sunset Smalls Bags 14 g Case 224 g,96.384615,127,58,13,128.076923


In [29]:
query_valuation_time_discount = ifa_query.create_company_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, .2)
pd.read_sql_query(query_valuation_time_discount, engine)

,total_inventory_valuation
0,111657.376717


In [28]:
query_valuation_time_discount = ifa_query.create_company_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, .2, False)
pd.read_sql_query(query_valuation_time_discount, engine)

,total_inventory_valuation
0,77038.683533


In [19]:
query_join = ifa_query.create_inventory_days_since_package_metric_query_by_product_name(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_join, engine)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,None,None,None,None,NaT,NaT,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,24704610,1A4060300022B79000319209,active,2022-04-28,2022-04-30 02:29:14+00:00,Product,"Crumble Green Jack 1g (S), Hashtag Distributio...",Extract (weight - each),12.0,Each,4386902,Concentrate,,31022CRUMGJ,"Room 12 Green Crack 1-11-21, Room 5 Jack Cake ...",False,False,False,None,None,NaN,2
1,None,None,None,None,NaT,NaT,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,24704284,1A4060300022B79000319227,active,2022-04-28,2022-04-30 02:29:14+00:00,Product,Legend OG Live Distillate 1g 510 Vape Cart (I)...,Vape Cartridge (weight - each),36.0,Each,4463119,Concentrate,,40522LOGLRD,"01/10/21, 10KGREENHOUSE_CHILI VERDE, 10KGREENH...",False,False,False,None,None,NaN,2
2,None,None,None,None,NaT,NaT,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,24704709,1A4060300022B79000320200,active,2022-04-28,2022-04-30 01:53:39+00:00,Product,"Buddies THC 50mg Capsule 2pc, Hashtag Distribu...",Capsule (weight - each),120.0,Each,4099941,Concentrate,,30322GELCAP50,"DSD-10.15.20-HV1317, DSD-11.20.20-HV1341, DSD-...",False,False,False,None,None,NaN,2
3,None,None,None,None,NaT,NaT,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,24704748,1A4060300022B79000319192,active,2022-04-28,2022-04-30 02:29:14+00:00,Product,"Crumble Blue Dream 1g (S), Hashtag Distributio...",Extract (weight - each),24.0,Each,2736209,Concentrate,,12622CRUMBD,Room 7 Blue Dream 11-15-21,False,False,False,None,None,NaN,2
4,None,None,None,None,NaT,NaT,NaN,NaN,None,None,None,None,None,None,NaN,NaN,NaN,24704553,1A4060300022B79000319223,active,2022-04-28,2022-04-30 02:29:14+00:00,Product,"White Widow CDT Dist 1g Vape (H), Hashtag Dist...",Vape Cartridge (weight - each),24.0,Each,3129062,Concentrate,,32922CDTWW,"01/10/21, 10KGREENHOUSE_CHILI VERDE, 10KGREENH...",False,False,False,None,None,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6570,C9-0000370-LIC,0104878644,active,Consumer,2021-06-02 15:09:04+00:00,2021-06-01 00:00:00+00:00,6.0,150.0,active,13568969,1A4060300006E2E000078850,KE - THC Sativa Cherry 100mg - 43g,Edible (weight - each),Each,1.0,20.0,20.0,7628120,1A4060300006E2E000042531,active,2020-09-01,2020-09-12 22:58:06+00:00,Product,KE - THC Sativa Cherry 100mg - 43g,Edible (weight - each),0.0,Each,383495,InfusedEdible,,,"1st, 2019-10-18-2-H, 2019-10-18-3-H, 2019-10-1...",False,False,False,None,None,274.0,11
6571,C9-0000370-LIC,0057431806,active,Consumer,2020-10-27 17:05:55+00:00,2020-10-01 00:00:00+00:00,10.0,377.0,active,7941086,1A4060300006E2E000044383,KE - THC Sativa Cherry 100mg - 43g,Edible (weight - each),Each,2.0,30.0,15.0,7628120,1A4060300006E2E000042531,active,2020-09-01,2020-09-12 22:58:06+00:00,Product,KE - THC Sativa Cherry 100mg - 43g,Edible (weight - each),0.0,Each,383495,InfusedEdible,,,"1st, 2019-10-18-2-H, 2019-10-18-3-H, 2019-10-1...",False,False,False,None,None,56.0,11
6572,C9-0000370-LIC,0158194658,active,Consumer,2022-01-16 20:52:26+00:00,2022-01-01 00:00:00+00:00,3.0,60.0,active,16926649,1A4060300006E2E000097381,KE - THC Sativa Cherry 100mg - 43g,Edible (weight - each),Each,1.0,20.0,20.0,7628120,1A4060300006E2E000042531,active,2020-09-01,2020-09-12 22:58:06+00:00,Product,KE - THC Sativa Cherry 100mg - 43g,Edible (weight - each),0.0,Each,383495,InfusedEdible

In [38]:
query_join = ifa_query.create_inventory_days_since_package_metric_query(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE)
pd.read_sql_query(query_join, engine)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C9-0000370-LIC,0185025661,active,Consumer,2022-05-04 13:58:00+00:00,2022-05-01 00:00:00+00:00,7,265.0,active,24704603,1A4060300022B79000320206,"BBrand Raspberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704603,1A4060300022B79000320206,active,2022-04-28,2022-05-05 11:18:17+00:00,Product,"BBrand Raspberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),10.0,Each,3389930,Concentrate,,41222BDTRAS,"0649_SFV OG 12-07, 10-15 peanut butter souffle...",False,False,False,None,None,6,7
1,C9-0000370-LIC,0185025661,active,Consumer,2022-05-04 13:58:00+00:00,2022-05-01 00:00:00+00:00,7,265.0,active,24704619,1A4060300022B79000320202,"BBrand Blueberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704619,1A4060300022B79000320202,active,2022-04-28,2022-05-05 11:18:18+00:00,Product,"BBrand Blueberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),22.0,Each,3389918,Concentrate,,31822BDTBLU,"1-21, 10-15 peanut butter souffle, 10-6 cherry...",False,False,False,None,None,6,7
2,C9-0000370-LIC,0184086989,active,Consumer,2022-04-30 16:29:57+00:00,2022-04-01 00:00:00+00:00,1,65.0,active,24704654,1A4060300022B79000319194,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),Each,1.0,65.0,65.0,24704654,1A4060300022B79000319194,active,2022-04-28,2022-05-02 11:16:41+00:00,Product,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),21.0,Each,3205886,Concentrate,,13122LDSCAP25DUO,"Amnesia FF, Harvest FF, PY-09.30.20-AG0671, PY...",False,False,False,None,None,2,4
3,C9-0000370-LIC,0185025665,active,Consumer,2022-05-04 13:41:47+00:00,2022-05-01 00:00:00+00:00,2,75.0,active,24704734,1A4060300022B79000319200,"Buddies THC 10mg Capsule 60pc, Hashtag Distrib...",Capsule (weight - each),Each,1.0,50.0,50.0,24704734,1A4060300022B79000319200,active,2022-04-28,2022-05-05 11:18:17+00:00,Product,"Buddies THC 10mg Capsule 60pc, Hashtag Distrib...",Capsule (weight - each),35.0,Each,2379103,Concentrate,,22522GELCAP10,"Amnesia FF, Clem FF, Clem FF1, CLEMFF, Harvest...",False,False,False,None,None,6,7
4,C9-0000370-LIC,0184086938,active,Consumer,2022-04-30 14:55:21+00:00,2022-04-01 00:00:00+00:00,3,156.0,active,24704751,1A4060300022B79000319190,Watermelon Zinn Live Distillate 1g 510 Vape Ca...,Vape Cartridge (weight - each),Each,1.0,40.0,40.0,24704751,1A4060300022B79000319190,active,2022-04-28,2022-05-06 11:18:25+00:00,Product,Watermelon Zinn Live Distillate 1g 510 Vape Ca...,Vape Cartridge (weight - each),21.0,Each,4168985,Concentrate,,11922WZLRD,"""33"" 12-12-19, #19, #20, #21, #22, #24, #25, #...",False,False,False,None,None,2,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900,C9-0000370-LIC,0132340471,active,Consumer,2021-09-27 13:17:08+00:00,2021-09-01 00:00:00+00:00,3,67.8,active,7903512,1A406030000A2C9000007832,Kushy Punch Recover Gummy,Edible (weight - each),Each,1.0,12.8,12.8,7903512,1A406030000A2C9000007832,active,2020-09-14,2022-03-17 11:18:48+00:00,Product,Kushy Punch Recover Gummy,Edible (weight - each),4.0,Each,1329434,InfusedEdible,,,,False,False,False,None,None,378,549
1901,C9-0000370-LIC,0161318645,active,Consumer,2022-01-29 15:42:47+00:00,2022-01-01 00:00:00+00:00,7,96.8,active,790

# Analyzer

In [21]:
analyzer = ifa.Analyzer()

In [25]:
analyzer.update_class_attributes(['EL'], ['C9-0000370-LIC'], True)

In [26]:
analyzer.find_most_valuable_products('tx_product_name', 'avg_days_since_sale', 'ASC', 10)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,Surplus: 1g Cart: Blueberry Cookies [I],15.666667,30,5,15,40.000000
1,Surplus: 1g Cart: Purple Lemonade [S],15.684211,28,3,19,40.000000
2,Surplus: 1g Cart: Trainwreck [H],17.228571,35,3,35,39.885714
3,HONEYLEAF - 3.5G INDICA,19.500000,34,5,16,24.687500
4,Surplus Tropics: 1g Cart: Strawberry Daze [S],20.950000,31,7,20,40.000000
5,Almora Farm: 3.5g Flower: Blue Banana [H],26.583333,35,18,24,25.000000
6,The Pairist - |PREROLL| 5 Pack (2.5g) Infused ...,27.384615,71,4,13,45.000000
7,HONEYLEAF - 3.5G SATIVA,27.466667,43,2,15,24.833333
8,Surplus Tropics: 1g Cart: Kiwi Blast [H],28.181818,35,22,11,40.000000
9,KE - CBN CBD Tranquility 1:1:1 - 40g,32.375000,68,12,16,25.000000


In [32]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price,sale_velocity_per_day
0,Surplus: 1g Cart: Trainwreck [H],18.297297,37,3,37,39.891892,2.022157
1,Surplus: 1g Cart: Blueberry Cookies [I],15.666667,30,5,15,40.000000,0.957447
2,Surplus Tropics: 1g Cart: Strawberry Daze [S],20.950000,31,7,20,40.000000,0.954654
3,Almora Farm: 3.5g Flower: Blue Banana [H],26.960000,36,18,25,25.000000,0.927300
4,HONEYLEAF - 3.5G INDICA,19.500000,34,5,16,24.687500,0.820513
5,1g Jar THC A - Strawberry Banana,48.105263,89,8,38,20.000000,0.789934
6,Humboldts Finest | Packaged Product | Legend O...,43.121212,86,8,33,1.000000,0.765285
7,BeMore Blunt Lemon Bean x Muffinz Sauce (1.5g)...,59.285714,125,7,35,29.914286,0.590361
8,Trifi Cookies-1g Pre-roll,39.304348,61,3,23,14.934783,0.585177
9,FloraCal Sunset Sherb (1g) Preroll,39.434783,75,19,23,1.173913,0.583241


In [35]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15)

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price,sale_velocity_per_day
0,Vape Cartridge (volume - each),Surplus: 1g Cart: Trainwreck [H],18.297297,37,3,37,39.891892,2.022157
1,Vape Cartridge (volume - each),Surplus: 1g Cart: Blueberry Cookies [I],15.666667,30,5,15,40.000000,0.957447
2,Vape Cartridge (volume - each),Surplus Tropics: 1g Cart: Strawberry Daze [S],20.950000,31,7,20,40.000000,0.954654
3,Flower (packaged eighth - each),Almora Farm: 3.5g Flower: Blue Banana [H],26.960000,36,18,25,25.000000,0.927300
4,Flower (packaged - each),HONEYLEAF - 3.5G INDICA,19.500000,34,5,16,24.687500,0.820513
5,Extract (weight - each),1g Jar THC A - Strawberry Banana,48.105263,89,8,38,20.000000,0.789934
6,Flower (packaged gram - each),Humboldts Finest | Packaged Product | Legend O...,43.121212,86,8,33,1.000000,0.765285
7,Pre-Roll Infused,BeMore Blunt Lemon Bean x Muffinz Sauce (1.5g)...,59.285714,125,7,35,29.914286,0.590361
8,Pre-Roll Leaf,Trifi Cookies-1g Pre-roll,39.304348,61,3,23,14.934783,0.585177
9,Pre-Roll Flower,FloraCal Sunset Sherb (1g) Preroll,39.434783,75,19,23,1.173913,0.583241


In [56]:
df_v1 = analyzer.get_inventory_valuation_query('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank
0,11497141,25.60,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1
...,...,...,...,...,...,...,...,...
378,8165921,NaN,0.0,Honeyleaf Cartridge - OG Kush - 0.5g,Vape Cartridge (volume - each),0.290411,NaN,1
379,8486461,23.04,4.0,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),1.405479,92.16,1
380,9166114,64.00,1.0,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.358904,64.00,1
381,9166120,55.00,2.0,N5 - Harmony 15ml - 15ml,Tincture (volume - each),1.304110,110.00,1


In [55]:
df_v2 = analyzer.get_inventory_valuation_query('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,avg_price_per_unit
0,26.420454,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333
...,...,...,...,...,...,...,...
378,NaN,8165921,Honeyleaf Cartridge - OG Kush - 0.5g,Vape Cartridge (volume - each),0.0,0.290411,NaN
379,27.095957,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,35.010000
380,54.118077,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,69.333333
381,22.074812,9166120,N5 - Harmony 15ml - 15ml,Tincture (volume - each),2.0,1.304110,28.000000


In [70]:
df_v1, valuation_1 = analyzer.find_inventory_valuation('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1.head()

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank,product_type,shelf_life,expired
0,11497141,0.00,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1,Concentrates,1.0,True
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1,Concentrates,1.0,False
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1,Edibles,1.0,False
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1,Concentrates,1.0,False
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1,Concentrates,1.0,False


In [71]:
valuation_1

111097.6

In [72]:
df_v1.shelf_life.value_counts()

1.0    325
0.5     58
Name: shelf_life, dtype: int64

In [73]:
df_v2, valuation_2 = analyzer.find_inventory_valuation('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727,Concentrates,1.0,True
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333,Concentrates,1.0,False
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667,Edibles,1.0,False
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000,Concentrates,1.0,False
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333,Concentrates,1.0,False


In [74]:
valuation_2

108088.96025899313